In [1]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import display, Image
import shutil
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
with open('test_ids.txt', 'r') as file:
    # Read all lines into a list
    lines = file.readlines()
    test_ids = []

    # Print each line
    for line in lines:
        test_ids.append(line.strip() + '.jpg')

In [6]:
"""with open('train_ids.txt', 'r') as file:
    # Read all lines into a list
    lines = file.readlines()
    train_ids = []

    # Print each line
    for line in lines:
        train_ids.append(line.strip() + '.jpg')"""

df = pd.read_csv("train_text.csv")
pd.set_option('display.float_format', '{:.0f}'.format)
train_ids = []
df

,id,tweet_text,Hate
0,1115290475811037185,@iSpotlight6800 Nigga said Prom Season 😂😂😂😂,0
1,1108180634730745857,idk why mither ben bizzy thought this was a lo...,0
2,1064098809314230272,😂😂😂😂😂 fs Jackie ya mad cunt,0
3,1113434967588188166,FullyVers nigga,0
4,1114301234767368192,Ayo fr who the fuck is this nigga,0
...,...,...,...
58949,1044597935005216768,When she rolled her sleeves up I knew this int...,1
58950,1056595445215059969,New Video: Spice – Black Hypocrisy,1
58951,1062161948748513280,@MSNBC @KellyannePolls Ann shut up you mentall...,1
58952,1105465552544374786,@quisLaFlare Good luck my nigga 🤘🏾,1


In [4]:
with open('val_ids.txt', 'r') as file:
    # Read all lines into a list
    lines = file.readlines()
    val_ids = []

    # Print each line
    for line in lines:
        val_ids.append(line.strip() + '.jpg')

In [5]:
print("Number of images in training dataset: ", len(train_ids))
print("Number of images in validation dataset: ", len(val_ids))
print("Number of images in testing dataset: ", len(test_ids))

Number of images in training dataset:  134823
Number of images in validation dataset:  5000
Number of images in testing dataset:  10000


In [6]:
#get image files
def get_image_files(folder_path):
    files = os.listdir(folder_path)
    image_files = []
    
    for file in files:
        if file.lower().endswith('.jpg'):
            image_files.append(file)
    
    return image_files
        

In [7]:
train_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/train'
test_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/test'
val_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/valid'

In [8]:
def copy_files(source_path, destination_path, titles):
    for title in titles:
        source_file = os.path.join(data_directory, title)
        destination_file = os.path.join(destination_path, title)
        shutil.copy(source_file, destination_file)

In [9]:
#data_directory = "/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/img_resized/"

In [10]:
#already done
#copy_files(data_directory, train_path, train_ids)
#copy_files(data_directory, val_path, val_ids)
#copy_files(data_directory, test_path, test_ids)

In [11]:
len(get_image_files(train_path))

134823

In [12]:
import json

# Specify the path to your JSON file
json_file_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/MMHS150K_GT.json'

# Open the JSON file for reading
with open(json_file_path, 'r') as json_file:
    # Load the JSON data
    data = json.load(json_file)

In [13]:
df = pd.DataFrame.from_dict(data, orient='index')
df = df.reset_index()
newColumns = {"index": "id"}
df = df.rename(columns=newColumns)

In [14]:
#0 for nothate, #1 for hate
def labels_to_binary(labels):
    if sum(labels) <= 1:
        return 0
    else:
        return 1

In [15]:
df["Hate(1)_vs_NotHate(0)"] = df["labels"].apply(labels_to_binary)
sum(df["Hate(1)_vs_NotHate(0)"]==1)

61547

In [16]:
nothate_ids = df[df["Hate(1)_vs_NotHate(0)"]==0]["id"].tolist()

In [17]:
hate_ids = df[df["Hate(1)_vs_NotHate(0)"]==1]["id"].tolist()

In [18]:
nothate_ids = [x + '.jpg' for x in nothate_ids]
hate_ids = [x + '.jpg' for x in hate_ids]

In [19]:
"""hate_train_ids = []
hate_val_ids = []
hate_test_ids = []
for id_ in hate_ids:
    if id_ in train_ids:
        hate_train_ids.append(id_)
    elif id_ in test_ids:
        hate_test_ids.append(id_)
    else:
        hate_val_ids.append(id_)""";

In [20]:
"""nothate_train_ids = []
nothate_val_ids = []
nothate_test_ids = []
for id_ in nothate_ids:
    if id_ in train_ids:
        nothate_train_ids.append(id_)
    elif id_ in test_ids:
        nothate_test_ids.append(id_)
    else:
        nothate_val_ids.append(id_)""";

In [21]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input

# Set the path to your dataset
dataset_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project'

# Define the image size based on the pre-trained model's requirements
img_size = (224, 224)

# Define the batch size
batch_size = 32

# Create an ImageDataGenerator for preprocessing images
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [22]:
# Create data generators for training, validation, and test sets
train_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming you have multiple classes
    shuffle=False  # Important: Set shuffle to False for feature extraction
)

validation_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, 'valid'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 134823 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


In [ ]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='/Users/raghavraahul/Downloads/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))

# Extract features from images using the pre-trained model
train_features = base_model.predict(train_generator, steps=len(train_generator))
validation_features = base_model.predict(validation_generator, steps=len(validation_generator))
test_features = base_model.predict(test_generator, steps=len(test_generator))

3975/4214 [===========================>..] - ETA: 1:08:57

In [ ]:
# Flatten the features
train_features = np.reshape(train_features, (len(train_generator.classes), -1))
validation_features = np.reshape(validation_features, (len(validation_generator.classes), -1))
test_features = np.reshape(test_features, (len(test_generator.classes), -1))

In [ ]:
#save for later use to train custom model
np.save('train_features.npy', train_features)
np.save('validation_features.npy', validation_features)
np.save('test_features.npy', test_features)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Load the extracted features
train_features = np.load('train_features.npy')
validation_features = np.load('validation_features.npy')
test_features = np.load('test_features.npy')

# Load the corresponding labels for training
train_labels = train_generator.classes
validation_labels = validation_generator.classes
test_labels = test_generator.classes

# Define and compile the custom model
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=train_features.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the extracted features
history = model.fit(train_features, train_labels, 
                    epochs=10, 
                    batch_size=32,
                    validation_data=(validation_features, validation_labels))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_features, test_labels)
print(f'Test accuracy: {test_acc * 100:.2f}%')